#Downloading raw read data from SRA

The below cell will download the raw Illumina data associated with the study and rename the files as specified in the text file `SraAccList.txt` provided in the `data` directory.

The cell expects modules of the [SRA Toolkit](http://www.ncbi.nlm.nih.gov/Traces/sra/?view=toolkit_doc) (`prefetch` and `fastq-dump`) to be in your path (tested with SRA Toolkit version 2.3.5). `prefetch` downloads data in sra format and places it per default into `~/ncbi/public/sra/`. `fastq-dump` converts the sra formatted data to gzipped fastq.

__removed again as I've not set it up correctly__

#Processing read data

To ensure complete reproducibility all subsequent processing of the read data was performed in a docker container which we have made available [here](https://hub.docker.com/r/chrishah/metabeat/).

The following cell will print out the help file for metaBEAT_global.py allowing readers to follow the commands used below.

In [ ]:
%%bash

metaBEAT_global.py -h

__PERFORM A CLUSTERING ANALYSIS TO CHECK FOR OPTIMAL CLUSTER READ DEPTH__

We will ultimately be clustering and filtering our data to unique sequences but we don't know what read depth we should accept as a minimum valid cluster size. To investigate this we will iterate across different minimum read depths and examine the effect this has on clusters retained for taxonomic assignment.

In [1]:
%%bash

# Move to the correct folder
mkdir -p ../Processed_data/Cluster_size
cd ../Processed_data/Cluster_size

# Loop across minimum cluster sizes
for i in $(seq 1 5 101)
do
      echo -e "running with coverage $i"
      metaBEAT_global.py -Q ../../data/OPM2_QueryMap.txt --cluster --merge --merged_only --length_filter 310 --product_length 400 --clust_match 1.0 --clust_cov $i --trim_minlength 100 -@ James.Kitson@newcastle.ac.uk -o metaBEAT &> cluster_log.txt
      mv metaBEAT_read_stats.csv metaBEAT_read_stats_$i.csv
done

# Combine the files into one large output for plotting in R
cat metaBEAT_read_stats_1.csv | head -n 1 > ../../data/combined_read_stats.csv
cat metaBEAT_read_stats_* | grep "sample," -v >> ../../data/combined_read_stats.csv
cd ..


running with coverage 1
running with coverage 6
running with coverage 11
running with coverage 16
running with coverage 21
running with coverage 26
running with coverage 31
running with coverage 36
running with coverage 41
running with coverage 46
running with coverage 51
running with coverage 56
running with coverage 61
running with coverage 66
running with coverage 71
running with coverage 76
running with coverage 81
running with coverage 86
running with coverage 91
running with coverage 96
running with coverage 101


__PERFORMING FINAL ANALYSES (INCL. TAXONOMIC ASSIGNMENT) FROM RAW READ DATA__

The following cell does the trimming and clustering then does a BLAST against a complete local copy of genbank using the metaBEAT pipeline.

In [14]:
%%bash

# Move to the correct folder
mkdir -p ../Processed_data/Final_analysis
cd ../Processed_data/Final_analysis

# Perform the clustering and assignment analyses with parameters determined above
metaBEAT_global.py -Q ../../data/OPM2_QueryMap.txt --cluster --merge --merged_only \
--length_filter 310 --product_length 400 --clust_match 1.0 --clust_cov 50 --trim_minlength 100 -E \
-n 6 -v --blast --blast_db ../../../../Genbank/nt/nt \
--min_ident 0.95 --min_ali_length 0.90 -@ James.Kitson@newcastle.ac.uk &> OPM2BLAST_log.txt


MetaBEAT produces files in the .biom format for easy use with QIIME but we will be using the output in R for plotting figures. The next cell strips out all the QIIME .biom formatting and leaves us with a rectangular read table.

In [1]:
%%bash
cd ../Processed_data/Final_analysis/

cat ./GLOBAL/BLAST_0.95/metaBEAT-by-taxonomy-readcounts.blast.tsv | \
grep "# " -v | sed 's/#//' | sed 's/\.blast//g' | sed 's/ /_/' | perl -ne \
'chomp; @a=split("\t"); pop(@a); $out=join("\t", @a); print "$out\n"' \
> ./GLOBAL/BLAST_0.95/metaBEAT-processed.tsv

The reformatted metaBEAT data could be used as is but we'll transpose our table now to make life easy.

In [2]:
import pandas as pd

filename = '../Processed_data/Final_analysis/GLOBAL/BLAST_0.95/metaBEAT-processed.tsv'
output_filename = '../data/metaBEAT_transpose.tsv'

df = pd.read_table(filename)
df = df.transpose()
df.to_csv(output_filename, header=None, sep='\t')